In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
#export
from fastai.torch_imports import *
from fastai.layers import *

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# default_exp layers

# Layers

> Some layers which tops up the ones in fastai

## BatchNorm layers

In [ ]:
#export
class Lin1BnDrop(nn.Sequential):
    "Module grouping `BatchNorm1d`, `Dropout` and a `Linear` layer with just one output feature"
    def __init__(self, n_in, n_out, bn=True, p=0., act=None, lin_first=False):
        layers = [BatchNorm(n_in, ndim=1)] if bn else []
        if p != 0: layers.append(nn.Dropout(p))
        lin = [nn.Linear(n_out, 1, bias=not bn)]
        if act is not None: lin.append(act)
        layers = lin+layers if lin_first else layers+lin
        super().__init__(*layers)

The `BatchNorm` layer is skipped if `bn=False`, as is the dropout if `p=0`. Optionally, you can add an activation for after the linear layer with act.

In [ ]:
tst = Lin1BnDrop(6594, 400)

In [ ]:
tst

Lin1BnDrop(
  (0): BatchNorm1d(6594, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=400, out_features=1, bias=False)
)

In [ ]:
tst = Lin1BnDrop(6594, 400, lin_first=True)
tst

Lin1BnDrop(
  (0): Linear(in_features=400, out_features=1, bias=False)
  (1): BatchNorm1d(6594, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_text.models.core.ipynb.
Converted 03_text.learner.ipynb.
Converted index.ipynb.
